# Example Evaluation Code

This notebook will be very __similar__ to the code I use to evaluate your results - it is provided for __your convenience__ so that you can use it to evaluate your preprocessing results at any time before your __final submission__.

Please note that the results here will __NOT__ be the same as my evaluation results.

Let's start with loading the required packages.

In [1]:
# import required package for data handling
import pandas as pd
import numpy as np

# import required packages for splitting data
from sklearn import model_selection
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

# import required packages for evaluating models
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_recall_fscore_support

# import `logistic regression` model
from sklearn.linear_model import LogisticRegression

Next you should load __your__ data. In this case, I am using a sample dataset (`GroupX.csv`) which contains 6 predictors (`X1 - X6`) and two target variables (`Y1, Y2`).

Please make sure you change the data to your __OWN__ dataset when using this code.

__NOTE__:
1. Your dataset maybe very different from the sample dataset.
2. Please follow this structure when submitting your dataset.

In [2]:
data = pd.read_csv('../csv_files/p2feature-selection.csv', header=0, index_col=0)
data.head()

,C1,C4,C6,C7,C5p,T4p,T3p,T5p,S1p,S2p,S3p,C2,C3p,Industry_Bins,Y1,Y2
0,0.030756,0.669243,0.343380,-0.477847,0.056127,0.168742,0.385070,0.310826,-0.146970,0.165628,-0.128062,1.0,0,1,0,1
1,1.279824,-0.643726,-1.645602,-0.789310,2.434275,0.229402,0.151494,-0.016168,-1.240880,0.781633,-0.147676,0.0,1,1,1,0
2,-0.431183,0.410546,-0.662792,-1.277684,-0.231083,0.118150,0.395935,1.107680,2.767209,-0.974221,0.044649,1.0,1,1,1,0
3,0.907828,0.389135,-0.803873,-1.226569,-0.274697,-1.243939,0.572453,1.094210,2.091907,-0.602916,0.697829,1.0,1,1,1,1
4,-0.603838,-1.310426,0.007764,1.026514,-0.020075,-0.574282,-0.085718,-0.357465,-0.364704,0.449384,-0.006421,1.0,0,3,0,1


In [3]:
data1 = data.drop(labels=['C5p', 'C1', 'C6', 'C7', 'T3p', 'T5p', 
                          'S1p', 'S3p','Industry_Bins' ], axis=1)
data1.head()

,C4,T4p,S2p,C2,C3p,Y1,Y2
0,0.669243,0.168742,0.165628,1.0,0,0,1
1,-0.643726,0.229402,0.781633,0.0,1,1,0
2,0.410546,0.118150,-0.974221,1.0,1,1,0
3,0.389135,-1.243939,-0.602916,1.0,1,1,1
4,-1.310426,-0.574282,0.449384,1.0,0,0,1


In [6]:
data2 = data.drop(labels=['C1', 'C6', 'C7', 'T3p', 'T4p','T5p','S1p','S3p','C3p'], axis=1)
data2.head()

,C4,C5p,S2p,C2,Industry_Bins,Y1,Y2
0,0.669243,0.056127,0.165628,1.0,1,0,1
1,-0.643726,2.434275,0.781633,0.0,1,1,0
2,0.410546,-0.231083,-0.974221,1.0,1,1,0
3,0.389135,-0.274697,-0.602916,1.0,1,1,1
4,-1.310426,-0.020075,0.449384,1.0,3,0,1


Checking your data types and make sure it follows the data dictionary would be an important step, you can do that using the `.dtypes` attribute.

__NOTE__: all __continuous__ faetures will be in `float64` data type, and all __categorical__ features will be in `int64` data type (given you already coded (per __suggest task \#6__ in the competition document) them).

In [7]:
data1.dtypes

C4     float64
T4p    float64
S2p    float64
C2     float64
C3p      int64
Y1       int64
Y2       int64
dtype: object

In [8]:
data2.dtypes

C4               float64
C5p              float64
S2p              float64
C2               float64
Industry_Bins      int64
Y1                 int64
Y2                 int64
dtype: object

Now you need to specify your targets and predictors. __NOTE__ we have two targets here (`Y1, Y2`).

In [9]:
y1 = data1.Y1
y2 = data2.Y2

Check the shape of the data.

In [10]:
data1.shape, data2.shape

((682, 7), (682, 7))

It is very possible that you will use different sets of the predictors for `Y1` and `Y2`. Now let's define them.

First, let's define predictors for `Y1` - which will be the first 5 features in `data`.

In [11]:
cols1 = list(data1.columns)
# first 5 features 
cols1[:-2]

['C4', 'T4p', 'S2p', 'C2', 'C3p']

Use below code to select the first 5 features as predictors for `Y1`.

In [12]:
predictors_y1 = data1[cols1[:-2]]
predictors_y1.head()

,C4,T4p,S2p,C2,C3p
0,0.669243,0.168742,0.165628,1.0,0
1,-0.643726,0.229402,0.781633,0.0,1
2,0.410546,0.118150,-0.974221,1.0,1
3,0.389135,-1.243939,-0.602916,1.0,1
4,-1.310426,-0.574282,0.449384,1.0,0


Upon investigation of the data, we know we have __six__ features (`X1 - X6`) predicting `Y2`. Use similar code (as below) to select them.

In [13]:
cols2 = list(data2.columns)
# first 5 features 
cols2[:-2]

['C4', 'C5p', 'S2p', 'C2', 'Industry_Bins']

In [14]:
predictors_y2 = data2[cols2[:-2]]
predictors_y2.head()

,C4,C5p,S2p,C2,Industry_Bins
0,0.669243,0.056127,0.165628,1.0,1
1,-0.643726,2.434275,0.781633,0.0,1
2,0.410546,-0.231083,-0.974221,1.0,1
3,0.389135,-0.274697,-0.602916,1.0,1
4,-1.310426,-0.020075,0.449384,1.0,3


Below is the key part of this notebook - which generates a `logistic regression` model to predict `Y1`/`Y2`.

The code works this way:

1. We generate two lists `f1_score_lst` and `auc_lst` to store f1_score and AUC from each of the `10` runs of the model;
2. Define model:
    1. We define a `LogisticRegression()` model;
    
    2. We split predictors (`predictors_y1`) and target `y1` to training (80%) and testing (20%);
    
    3. We fit the model `clf` to the training data, then use it to predict on the testing data;
    
    4. We also defined a `10-fold cross validation` to make sure our model do not overfit - see [here](https://scikit-learn.org/stable/modules/cross_validation.html) for more info;
    
    5. We append the f1_score and AUC of current model to the lists (`f1_score_lst` and `auc_lst`) we defined earlier.
  
3. Print out average f1_score and AUC for all 10 runs;
4. Print out average average accuracy from cross validation
5. Print out confusion matrix and classification report for the __last__ model.

__NOTE__: Step 3 provides the evaluation results we need; step 4 - 5 can be used to verify the results from step 3.

In [15]:
# lists for f1-score and AUC
f1_score_lst = []
auc_lst = []


#loop to calculate f1 and auc scores and present averages after 10 runs
for count in range (1,10):
    #Model building
    clf = LogisticRegression()
    X1_train, X1_test, y1_train, y1_test = train_test_split(predictors_y1, y1, test_size=0.2, random_state=123)
    clf.fit(X1_train, y1_train)

    y1_pred = clf.predict(X1_test)

    
    #10-fold cross validation
    kfold = model_selection.KFold(n_splits=10, random_state=7)
    scoring = 'accuracy'
    results = model_selection.cross_val_score(clf, X1_train, y1_train, cv=kfold, scoring=scoring)

    

    
    #calculate f1-score and AUC
    
    clf_roc_auc = roc_auc_score(y1_test, y1_pred)
    f1_score_lst.append(precision_recall_fscore_support(y1_test, y1_pred, average='weighted')[2])
    auc_lst.append(clf_roc_auc)


print('F1 {:.4f}; AUC {:.4f} '.format(np.mean(f1_score_lst),np.mean(auc_lst)))

#result=logit_model.fit()
confusion_matrix_y1 = confusion_matrix(y1_test, y1_pred)


#print(result.summary())
print('Accuracy of classifier on test set: {:.2f}'.format(clf.score(X1_test, y1_test)))

print("10-fold cross validation average accuracy of classifier: %.3f" % (results.mean()))

print('Confusion Matrix for Logistic Regression Classfier:')
print(confusion_matrix_y1)

print('Classification Report for Logistic Regression Classfier:')
print(classification_report(y1_test, y1_pred))


F1 0.5692; AUC 0.5678 
Accuracy of classifier on test set: 0.57
10-fold cross validation average accuracy of classifier: 0.578
Confusion Matrix for Logistic Regression Classfier:
[[35 30]
 [29 43]]
Classification Report for Logistic Regression Classfier:
             precision    recall  f1-score   support

          0       0.55      0.54      0.54        65
          1       0.59      0.60      0.59        72

avg / total       0.57      0.57      0.57       137



Below code are used to evaluate model toward `Y2`. It is very similar to the code above - key difference is that `Y2` is imbalanced - so I wrote some code (under `# Begin oversampling`) to deal with that.

In [16]:
# lists for f1-score and AUC
f1_score_lst = []
auc_lst = []


#loop to calculate f1 and auc scores and present averages after 10 runs
for count in range (1,10):
    #Model building
    clf1 = LogisticRegression()

    
    # Splitting data into testing and training
    X2_train, X2_test, y2_train, y2_test = train_test_split(predictors_y2, y2, test_size=0.2, random_state=123)
    
    # Begin oversampling
    oversample = pd.concat([X2_train,y2_train],axis=1)
    max_size = oversample['Y2'].value_counts().max()
    lst = [oversample]
    for class_index, group in oversample.groupby('Y2'):
        lst.append(group.sample(max_size-len(group), replace=True))
    X2_train = pd.concat(lst)
    y2_train=pd.DataFrame.copy(X2_train['Y2'])
    del X2_train['Y2']
    
    # fitting model on oversampled data
    clf1.fit(X2_train, y2_train)
    
    y2_pred = clf1.predict(X2_test)
    
    
    #10-fold cross validation
    kfold = model_selection.KFold(n_splits=10, random_state=123)
    scoring = 'accuracy'
    results = model_selection.cross_val_score(clf1, X2_train, y2_train, cv=kfold, scoring=scoring)
    
    #calculate f1-score and AUC
    
    clf1_roc_auc = roc_auc_score(y2_test, y2_pred)
    
    
    #calculate average f1-score and AUC
    f1_score_lst.append(precision_recall_fscore_support(y2_test, y2_pred, average='weighted')[2])
    auc_lst.append(clf1_roc_auc)
    
    
print('F1 {:.4f}; AUC {:.4f} '.format(np.mean(f1_score_lst),np.mean(auc_lst)))

confusion_matrix_y2 = confusion_matrix(y2_test, y2_pred)


print('Accuracy of classifier on test set: {:.3f}'.format(clf1.score(X2_test, y2_test)))

print("10-fold cross validation average accuracy of clf1: %.3f" % (results.mean()))

print('Confusion Matrix for Classfier:')
print(confusion_matrix_y2)

print('Classification Report for Classfier:')
print(classification_report(y2_test, y2_pred))


F1 0.6114; AUC 0.5749 
Accuracy of classifier on test set: 0.606
10-fold cross validation average accuracy of clf1: 0.487
Confusion Matrix for Classfier:
[[25 19]
 [35 58]]
Classification Report for Classfier:
             precision    recall  f1-score   support

          0       0.42      0.57      0.48        44
          1       0.75      0.62      0.68        93

avg / total       0.65      0.61      0.62       137

